## YOLO
- YOLO는 you only look once, 즉 이미지를 한번 보는 것만으로 Obejct의 종류와 위치를 추측하는 딥러닝 기반의 물체인식 알고리즘
- 1-stage Detector로서 간단한 처리과정으로 다른 딥러닝 알고리즘에 비해 속도가 매우 빠르며, 높은 mAP(mean Average Precision)을 보인다 (단, 작은 Object)에 대해서는 낮은 정확도를 보인다
- Fast R-CNN과 비교했을 때 훨씬 적은 False-Positive를 보여준다 (low-background error)

![yolo_image](img/yolo_image.jpg)

![YOLO](img/YOLO.PNG)

### ■ YOLO 순서
- image Resizing
- Run Convolutional Netwrok
- Non-max suppression

### ■ YOLO의 장점과 단점

- 장점
    - 매우 빠른 속도와 복잡하지 않은 파이프라인
    - Image를 바라볼 때 전체를 한 번에 바라보는 방식으로 class에 대한 맥락적 이해도가 높다. (globally 하고) 
    - 이로인해 Background error (False-Positive) 아닌데 맞다고 하는 경우가 적 (Fast R-CNN과 비교했을 때, background error가 절반 정도) 
    - object의 일반적인 특징을 학습한다. (일반화에 강하여, 새로운 도메인이나 예상하지 못한 input이 들어와도 망가질 확률이 적다).

- 단점
    - 작은 Obejct에 대해 상대적으로 낮은 정확도를 보인다.

### ■ YOLO Network (1)

![yolo_network](img/yolo_network.jpg)

- YOLO의 Network Architecture는 GoogLeNet for image classification 모델을 기반
- 24 Convolutional layers & 2 Fully Connected layers

<hr>

### ■ YOLO Network (2)
- 7x7은 49개의 grid cell을 의미하며 각각의 Grid Cell은 B개의 Bounding Box를 가지고 있다. (B=2) 
- 앞 5개의 값은 해당 Grid Cell의 첫번째 Bounding Box에 대한 값이 채워져있다.

![yolo_network2](img/yolo_network2.jpg)

<hr>

### ■ YOLO Network (3)
- 6~10번째 값은 두 번째 Bounding Box에 대한 내용
![yolo_network3](img/yolo_network3.jpg)

<hr>

### ■ YOLO Network (4)
- 나머지 20개의 값은 20개의 class에 대한 conditional class probability에 해당한다
![yolo_network4](img/yolo_network4.jpg)

<hr>

### ■ YOLO Network (5)
- **Confidence score * 각 conditional class probability = class specific confidence score**
- 첫번째 Bounding box의 confidence score와 각 conditional class probability를 곱하면 첫번째 Bounding box의 class specific confidence score가 나온다.
- 두번쨰 Bounding box의 confidence score와 각 conditional class probability를 곱하면 두번째 Bounding box의 class specific confidence score가 나온다

- 총 Bounding box에 98개의 class specific confidence score를 얻을 수 있음
![yolo_network5](img/yolo_network5.jpg)

<hr>

### ■ YOLO Network 최종
- 98개의 class specific confidence score에 대해 각 20개의 클래스를 기준으로 non-maximum suppression을 진행
- 최종적으로 Object에 대한 Class 및 Bounding box Location을 결정한다

<hr>

### ■ NMS 알고리즘
- Non Maximum Suppression 으로 Leave only one Bounding Box
- 바운딩 박스가 많을 때 최종 하나만 남기게 되는 것
- Bounding Box에서 threshold 이하의 값들은 다 버리고 정렬을 진행했을 때, IOU 값이 0.5가 넘으면 그다음 확률들을 나머지를 0으로 만듬
- 해당과정을 거치면서 최종 Bounding Box를 만들어냄.

<hr>

### ■ YOLO Loss Function
- Object가 있는 경우가 없는 경우보다 큰 loss를 준다 (감마 coord=5, 감마 noobj=0.5) 
- 첫번쨰줄이 위치에 대한 loss / 두번째줄이 크기에 대한 loss
- 큰 물체에 대해서는 루트를 씌워서 영향을 조금 줄이고,
![yolo_loss_function](img/yolo_loss_function.jpg)

<hr>

### ■ YOLO 한계점 
- Yolo는 아무런 기본 값없이 2개의 Box를 예측하도록 함
- 각각의 grid cell이 하나의 클래스만을 예측가능, 작은 object 여러개가 다닥다닥 붙으면 제대로 예측 불가
- bounding box의 형태가 training data를 통해서만 학습되므로, 새로운/독특한 형태의 bouding box의 경우 정확히 예측하지 못한다.
- 몇 단계의 layer를 거쳐서 나온 feature map을 대상으로 bouding box를 예측하므로 localization이 다소 부정확해지는 경우가 있다.

## 참고문헌
- https://curt-park.github.io/2017-03-26/yolo/